In [1]:
import tensorflow as tf
import numpy as np

In [2]:
def ROIPoolingLayer(inp, roi, pooled_h, pooled_w):
    '''
        inp: input feature, N x H x W x C
        
        roi: region of interests (roi) bbox
             each one is [batch_idx, start_h, start_w, crop_h, crop_w]
             !!! must be a list
        
        pooled_h/pooled_w: target h/w
        
        out: output feature N x ph x pw x C
    '''
    input_shape = inp.get_shape().as_list()
    inp_h = input_shape[1]
    inp_w = input_shape[2]
    
    roi_num = len(roi)
    
    outlist = []
    for i in xrange(roi_num):
        btch_i, st_h, st_w, cp_h, cp_w =  roi[i]
        
        pool_sz_h = cp_h // pooled_h
        pool_sz_w = cp_w // pooled_w
        
        real_cp_h = pooled_h * pool_sz_h
        real_cp_w = pooled_w * pool_sz_w
        
        cropped_feature = tf.slice(inp, [btch_i, st_h, st_w, 0], [1, real_cp_h, real_cp_w, -1])
        
        outlist.append(tf.nn.max_pool(cropped_feature, ksize=[1, pool_sz_h, pool_sz_w, 1], \
                                      strides=[1, pool_sz_h, pool_sz_w, 1], padding='SAME'))
    
    return tf.concat(0, outlist)

In [3]:
inp = np.zeros((2, 12, 12, 2))

a = np.ones((6, 4))

b = np.hstack((a, a*2))

b = np.hstack((b, a*3))

b = np.vstack((b, b*2))

c = np.asarray([b, b*1.5])

c = np.transpose(c, [1, 2, 0])

inp[0] = c
inp[1] = c+1

roi = [[0, 6, 8, 6, 4], [1, 0, 0, 8, 5], [0, 3, 3, 4, 4]]

In [4]:
sess = tf.InteractiveSession()
inp = tf.Variable(inp, dtype=tf.float32)

sess.run(tf.initialize_all_variables())

In [5]:
out = ROIPoolingLayer(inp, roi, pooled_h=3, pooled_w=3)

In [7]:
o = sess.run(out)

In [8]:
o.shape

(3, 3, 3, 2)

In [9]:
o[0, :, :, 0]

array([[ 6.,  6.,  6.],
       [ 6.,  6.,  6.],
       [ 6.,  6.,  6.]], dtype=float32)

In [10]:
o[1, :, :, 1]

array([[ 2.5,  2.5,  2.5],
       [ 2.5,  2.5,  2.5],
       [ 2.5,  2.5,  2.5]], dtype=float32)

In [11]:
o[2, :, :, 1]

array([[ 1.5,  3. ,  3. ],
       [ 1.5,  3. ,  3. ],
       [ 1.5,  3. ,  3. ]], dtype=float32)